In [ ]:
import ssl
import certifi
import pandas as pd
from indexMapping import indexMapping
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

ssl_context = ssl.create_default_context(cafile=certifi.where())
es = Elasticsearch("http://localhost:9200")
es.ping()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.fillna("None", inplace=True)

model = SentenceTransformer('all-mpnet-base-v2')
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

indexName = "all_products"
if es.indices.exists(index=indexName):
    es.indices.delete(index=indexName)
    
es.indices.create(index="all_products", mappings=indexMapping)
record_list = df.to_dict("records")

for record in record_list:
    try:
        es.index(index=indexName, document=record, id=record["ProductID"])
    except Exception as e:
        print(e)
        
es.count(index="all_products")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
input_keyword = "blue shose"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index= indexName, knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

/var/folders/4g/b79qqr8d0bv9k0z5rs48rmp40000gn/T/ipykernel_37978/3728189831.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index= indexName, knn=query , source=["ProductName","Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.45106566,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.45106566,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]